In [1]:
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.datasets import mnist


In [2]:
with open("generated_data.json", "r") as f:
    data = json.load(f)

In [3]:
X = []
y = []

for item in data:
    keypoints = item["keypoints"][0]
    X.append(np.array(keypoints).flatten())
    y.append(item["label"])
X = np.array(X)

In [4]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(y_encoded)

[0 0 0 ... 2 2 2]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()


In [6]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (32000, 34)
y_train shape: (32000,)
X_test shape: (8000, 34)
y_test shape: (8000,)


In [7]:
num_classes = len(label_encoder.classes_)
input_shape = X_train.shape[1]

In [8]:
X_train = X_train.reshape(-1, 17, 2, 1)
X_test= X_test.reshape(-1, 17, 2, 1)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

num_classes = len(label_encoder.classes_)
input_shape = (17, 2, 1)  # 高度、宽度、通道数

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# 编译模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/Users/mikeshinoda/Library/Caches/pypoetry/virtualenvs/yolo-pose-estimation-DqWaCygJ-py3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# model = Sequential()
# model.add(Dense(128, activation="relu", input_shape=(input_shape,)))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation="relu"))
# model.add(Dense(num_classes, activation="softmax"))

# model.compile(
#     optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
# )

In [11]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)


Epoch 1/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4993 - loss: 0.9449 - val_accuracy: 0.8503 - val_loss: 0.3247
Epoch 2/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8272 - loss: 0.3559 - val_accuracy: 0.8650 - val_loss: 0.2917
Epoch 3/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8549 - loss: 0.3032 - val_accuracy: 0.8816 - val_loss: 0.2566
Epoch 4/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 987us/step - accuracy: 0.8636 - loss: 0.2848 - val_accuracy: 0.8928 - val_loss: 0.2415
Epoch 5/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 983us/step - accuracy: 0.8735 - loss: 0.2749 - val_accuracy: 0.9013 - val_loss: 0.2266
Epoch 6/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 936us/step - accuracy: 0.8848 - loss: 0.2568 - val_accuracy: 0.9084 - val_loss: 0.2163
Epoch 7/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 953us/step - accuracy: 0.8846 - loss: 0.2585 - val_accuracy: 0.8631 - val_loss: 0.2878
Epoch 8/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step - accuracy: 0.8871 - loss: 0.2409 - val_ac

In [12]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("\n测试集准确率:", test_acc)

250/250 - 0s - 411us/step - accuracy: 0.9071 - loss: 0.2137

测试集准确率: 0.9071249961853027


In [13]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print("预测的类别：", y_pred_classes)
print("真实的类别：", y_test)
y_test_labels = label_encoder.inverse_transform(y_test)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

print(classification_report(y_test_labels, y_pred_labels))


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step
预测的类别： [2 1 3 ... 1 0 2]
真实的类别： [2 1 3 ... 1 0 2]
              precision    recall  f1-score   support

        jump       1.00      1.00      1.00      2047
         run       1.00      0.63      0.77      1967
       stand       1.00      1.00      1.00      2000
        walk       0.73      1.00      0.84      1986

    accuracy                           0.91      8000
   macro avg       0.93      0.91      0.90      8000
weighted avg       0.93      0.91      0.90      8000

